In [2]:
import sys
import numpy as np
import pandas as pd
import lightgbm as lg
from tqdm import tqdm
import pandas as pd
from pathlib import Path

# from importlib import reload


import catboost
from catboost.utils import get_gpu_device_count

In [31]:
SIDE_BID = 0 
SIDE_ASK = 1

print(f"curdir: {Path.cwd()}")


for fname, pickle_name, npz_name in zip(
        ["../../data/train_small_A.npz", "../../data/train_small_B.npz"],
        ["../../data/X_train_small_A.pickle", "../../data/X_train_small_B.pickle"],
        ["../../data/X_train_small_A.npz", "../../data/X_train_small_B.npz"]):
    arr = np.load(fname)
    X_df = pd.DataFrame(data=arr["events"], columns=arr["columns"])
    X_df = X_df.loc[X_df.is_snapshot == 0]
    X_df = X_df.loc[(X_df.action == 3) | (X_df.Y >= 0)]
    X_df.drop(X_df.loc[(X_df.action == 2) &
                       (X_df.type == 1)].index, axis=0, inplace=True)

    X_df.drop(X_df.loc[(X_df.action == 0) &
                       (X_df.type == 1)].index, axis=0, inplace=True)

    X_df.to_pickle(pickle_name)

    np.savez(npz_name, events=X_df.values, columns=X_df.columns)
    print(f"{pickle_name} shape after clearing snapsots", X_df.shape)

curdir: /home/sergey/mnt/st1500/Usr/Sergey/TheJob/Challenges/wunder_summer/wunder_challenge/examples/catboost_local
../../data/X_train_small_A.pickle shape after clearing snapsots (767493, 9)
../../data/X_train_small_B.pickle shape after clearing snapsots (784181, 9)


In [30]:

X_train = pd.read_pickle("../../data/X_train_small_A.pickle")
X_train.drop(labels=["time", "id", "is_snapshot"], inplace=True, axis=1)
Y_train = pd.read_pickle("../../data/Y_train_small_A.pickle")
print("X_train.value_counts()")
print(X_train[["action", "type", "side", "Y"]].value_counts())
print(X_train[["action", "Y"]].value_counts())

X_train.value_counts()
action  type  side  Y 
3       0     1     -1    270085
              0     -1    263949
              1      0     54488
              0      0     52122
1       1     0      0     43928
              1      0     38755
                     1     13612
3       0     0      1     12472
              1      1      9973
1       1     0      1      8109
dtype: int64
action  Y 
3       -1    534034
         0    106610
1        0     82683
3        1     22445
1        1     21721
dtype: int64


In [ ]:



X_test = pd.read_pickle("../../data/X_train_small_B.pickle")
X_test.drop(labels=["time", "id", "is_snapshot"], inplace=True, axis=1)
Y_test = pd.read_pickle("../../data/Y_train_small_B.pickle")

print(X_test.head())

print( "********* FINISH dataset collection ***********************")

In [ ]:
new_chunk_indexes = X_train[X_train.action == 10].index

def get_new_chunk(new_chunk_indexes):
    for start, end in zip(new_chunk_indexes[:-1],
                          new_chunk_indexes[1:]):
        print(start, end)
        print(start+1, end+1)
        print()
        yield start+1, end+1

#for start, end in get_new_chunk(new_chunk_indexes):
#    print(start, end, X_train.action.iloc[start], X_train.action.iloc[end])

In [ ]:
main_df = X_train
chunks = []
for start, end in get_new_chunk(new_chunk_indexes):
    chunks.append((start, end))
    print("X_train.iloc[start].name", X_train.iloc[start].name, X_train.iloc[start].action)
    print("X_train.iloc[end].name", X_train.iloc[end].name, X_train.iloc[end].action)

chunk_borders = chunks[1]

In [ ]:

chunk_df = X_train.iloc[chunk_borders[0]:chunk_borders[1]]
print(chunk_df[["action", "type", "side"]].value_counts())
features_pattern = np.full(shape=8, fill_value=np.nan)

print(chunk_borders)
print(chunk_df)

price_features = 0